##Libraries

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
#Import modules required by Acis
import urllib.request
import urllib.error
import json

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
import numpy as np
import pandas as pd
from IPython import display
from IPython.display import clear_output

In [ ]:
import tensorflow
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("TensorFlow version:", tensorflow.__version__)

Found GPU at: /device:GPU:0
TensorFlow version: 2.8.2


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 15 19:01:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |    375MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


##Get Data

In [ ]:
outfdr = "/content/drive/MyDrive/Modules/Module_30_Capstone_4_Final_project/weather_data/"

pfw_prepped = pd.read_csv(outfdr + "pfw_prepped.csv")
pfw_prepped = pfw_prepped.drop(columns = 'Unnamed: 0')

In [ ]:
# Bounding box
lat_min = pfw_prepped.latitude.min()
lat_max = pfw_prepped.latitude.max()
long_min = pfw_prepped.longitude.min()
long_max = pfw_prepped.longitude.max()
bbox = ', '.join(str(x) for x in [long_min, lat_min, long_max, lat_max])

# Date Ranges
split = 6
inc = (datetime.date.fromisoformat(pfw_prepped.datetime.max()) - datetime.date.fromisoformat(pfw_prepped.datetime.min())).days // split
date_ranges = []
for ii in np.arange(0, split):
    if ii in np.arange(0, split-1):
        start = (datetime.date.fromisoformat(pfw_prepped.datetime.min()) + datetime.timedelta(days=int(ii)*inc)).strftime("%Y%m%d")
        end = (datetime.date.fromisoformat(pfw_prepped.datetime.min()) + datetime.timedelta(days=(int(ii)+1)*inc)).strftime("%Y%m%d")
    else:
        start = (datetime.date.fromisoformat(pfw_prepped.datetime.min()) + datetime.timedelta(days=int(ii)*inc)).strftime("%Y%m%d")
        end = datetime.date.fromisoformat(pfw_prepped.datetime.max()).strftime("%Y%m%d")
    date_ranges.append([start, end])

In [ ]:
# ACIS-WS Sample Code
# Set ACIS data server
base_url = "http://data.rcc-acis.org/"
#######################################
# ACIS WebServices functions
#######################################
def make_request(url,params):
    req = urllib.request.Request(url,
    json.dumps(params).encode('utf-8'),
    {"Content-Type":"application/json"})
    try:
        response = urllib.request.urlopen(req)
        return json.loads(response.read())
    except urllib.error.HTTPError as error:
        if error.code == 400 : print(error.msg)

def GridData(params) :
    return make_request(base_url+"GridData",params)

def flatten(t):
    return [item for sublist in t for item in sublist]

for varb in ['pcpn', 'mint', 'maxt']:
    for dr in date_ranges:
        start = dr[0]   # Start date
        end = dr[1]   # End date
        outf = varb + '_' + start + '_' + end + '.csv'   # Filename of csv

        ###################################################
        # M A I N
        ###################################################
        if __name__ == "__main__":
            # Set parameters for data request
            params = {
                "bbox" : bbox,
                "sdate" : start,
                "edate" : end,
                "grid" : "1",
                "elems" : [varb],
                "meta" : "ll"
            }
            # Obtain data
            data = GridData(params)

        lat = flatten(data['meta']['lat'])
        lon = flatten(data['meta']['lon'])
        dfa = pd.DataFrame(columns = ['latitude', 'longitude', 'month', 'day', 'year', varb])

        for dd in np.arange(0, len(data['data'])):
            date = list(np.repeat(data['data'][dd][0], len(lon)))
            clear_output()
            print(varb)
            print(date[0])

            if date[0][5:7] in ['10', '11', '12']:
                month = [x[5:7] for x in date]
            else:
                month = [x[6:7] for x in date]

            if date[0][8:10] in ['01', '02', '03', '04', '05', '06', '07', '08', '09']:
                day = [x[9:10] for x in date]
            else:
                day = [x[8:10] for x in date]

            year = [x[0:4] for x in date]

            datfl = flatten(data['data'][dd][1])
            dct = {'latitude' : lat,
                  'longitude' : lon,
                  'month' : month,
                  'day' : day,
                  'year' : year,
                  varb : datfl}
        
            dfb = pd.DataFrame(dct)
            dfa = pd.concat([dfa, dfb], ignore_index=True)

        dfa.to_csv(outfdr + outf)

maxt
2021-04-30


##Concatenate Data

In [ ]:
for varb in ['pcpn', 'mint', 'maxt']:
    listy = []
    for dr in date_ranges:
        start = dr[0]   # Start date
        end = dr[1]   # End date
        outf = varb + '_' + start + '_' + end + '.csv'   # Filename of csv
        imprt = pd.read_csv(outfdr + outf)
        listy.append(imprt)
    dfc = pd.concat(listy, ignore_index=True)
    dfc = dfc.drop(columns = 'Unnamed: 0')
    if varb[3] == 't':
        dfc.to_csv(outfdr + varb[3] + varb[0:3] + '.csv')
    else:
        dfc.to_csv(outfdr + varb + '.csv')